# ETL

---

**Author:** Diego Antonio García Padilla

**Date:** Oct 29, 2025

## Enviroment setup

In [1]:
#@title Setup & Environment Verification

import warnings
warnings.filterwarnings('ignore')

import os
import sys

print("=== ENVIRONMENT CHECK ===")
print(f"Python: {sys.version.split()[0]}")
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"SPARK_HOME: {os.environ.get('SPARK_HOME')}")
print(f"Driver Memory: {os.environ.get('SPARK_DRIVER_MEMORY')}")
print(f"Executor Memory: {os.environ.get('SPARK_EXECUTOR_MEMORY')}")
print("=" * 50)

=== ENVIRONMENT CHECK ===
Python: 3.10.12
JAVA_HOME: /usr/lib/jvm/java-8-openjdk-arm64/jre
SPARK_HOME: /opt/spark
Driver Memory: 12g
Executor Memory: 8g


In [2]:
#@title Import Libraries

# PySpark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer

# SciKit Learn
from sklearn.model_selection import train_test_split

# Data manipulation
import pandas as pd
import numpy as np

# Financial data
import yfinance as yf

# Hugging Face
from huggingface_hub import hf_hub_download

# Kaggle
import kagglehub

# Utilities
from datetime import datetime, timedelta
import json
import requests
import logging
from tqdm import tqdm
import time
import subprocess
from pathlib import Path

In [3]:
#@title Start Spark session

print("=== PRE-FLIGHT CHECK ===")

# Verify Java is available
try:
    java_version = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)
    print("Java: ✅ Available")
except Exception as e:
    print(f"Java: ❌ Not available - {e}")

print("=" * 50)

# 🔥 STOP any existing Spark sessions first
try:
    SparkContext.getOrCreate().stop()
    print("🧹 Cleaned up existing Spark session")
except:
    print("🆕 No existing session to clean")

print("=" * 50)

# Create fresh Spark session
spark = SparkSession.builder \
    .appName("Yelp_Sentiment_Analysis") \
    .master("local[*]") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "16") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "512m") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✅ Spark session configured with:")
print(f"   - Driver Memory: 12GB")
print(f"   - Executor Memory: 8GB")
print(f"   - Max Result Size: 4GB")
print(f"   - Parallelism: 16 cores")
print(f"   - Shuffle Partitions: 200")

=== PRE-FLIGHT CHECK ===
Java: ✅ Available


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/06 17:41:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/06 17:41:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


🧹 Cleaned up existing Spark session
✅ Spark session configured with:
   - Driver Memory: 12GB
   - Executor Memory: 8GB
   - Max Result Size: 4GB
   - Parallelism: 16 cores
   - Shuffle Partitions: 200


25/11/06 17:41:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Download data

### Yelp Reviews

In [4]:
#@title Download Yelp reviews dataset

yelp_path = kagglehub.dataset_download("yelp-dataset/yelp-dataset")

print("Path to dataset files:", yelp_path)

def explore_dataset(path):
    print(f"📁 Content of: {path}\n")
    total_size = 0
    
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            size = os.path.getsize(file_path)
            total_size += size
            size_mb = size / (1024 * 1024)
            print(f"  📄 {file}")
            print(f"     Size: {size_mb:.2f} MB")
    
    total_gb = total_size / (1024 * 1024 * 1024)
    print(f"\n💾 Total size: {total_gb:.2f} GB ({total_size / (1024 * 1024):.2f} MB)")
    
    return total_size

explore_dataset(yelp_path)

Path to dataset files: /root/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4
📁 Content of: /root/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4

  📄 Dataset_User_Agreement.pdf
     Size: 0.08 MB
  📄 yelp_academic_dataset_business.json
     Size: 113.36 MB
  📄 yelp_academic_dataset_checkin.json
     Size: 273.67 MB
  📄 yelp_academic_dataset_review.json
     Size: 5094.40 MB
  📄 yelp_academic_dataset_tip.json
     Size: 172.24 MB
  📄 yelp_academic_dataset_user.json
     Size: 3207.52 MB

💾 Total size: 8.65 GB (8861.26 MB)


9291705417

## Data Explotation

In [5]:
#@title Load Yelp Reviews as Spark dataset

# Parquet path
parquet_path = "../data/raw/yelp_reviews_raw.parquet"

if os.path.exists(parquet_path):
    print(f"✅ Already exists: {parquet_path}")
    df_reviews = spark.read.parquet(parquet_path)
    print(f"🔄 Loaded \n")
else:
    # Load the dataset
    reviews_file = os.path.join(yelp_path, "yelp_academic_dataset_review.json")

    # Read JSON file with Spark
    df_reviews = spark.read.json(reviews_file)

    # Show schema to understand structure
    print("📋 Schema of Yelp Reviews:")
    df_reviews.printSchema()

    # Basic statistics
    print(f"\n📊 Total reviews: {df_reviews.count():,}")

    # Show sample data
    print("\n🔍 Sample reviews:")
    df_reviews.show(5, truncate=50)

    # Check stars distribution
    print("\n⭐ Stars distribution:")
    df_reviews.groupBy('stars').count().orderBy('stars').show()

    # Check text lengths

    print("\n📝 Text statistics:")
    df_reviews.select(
        F.avg(F.length(F.col('text'))).alias('avg_length'),
        F.min(F.length(F.col('text'))).alias('min_length'),
        F.max(F.length(F.col('text'))).alias('max_length')
    ).show()

    df_reviews.write.parquet(parquet_path, mode="overwrite")
    print(f"\n💾 Parquet saved: {parquet_path}")

📋 Schema of Yelp Reviews:
root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)




📊 Total reviews: 6,990,280

🔍 Sample reviews:
+----------------------+----+-------------------+-----+----------------------+-----+--------------------------------------------------+------+----------------------+
|           business_id|cool|               date|funny|             review_id|stars|                                              text|useful|               user_id|
+----------------------+----+-------------------+-----+----------------------+-----+--------------------------------------------------+------+----------------------+
|XQfwVwDr-v0ZS3_CbbE5Xw|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-VcAEodg|  3.0|If you decide to eat here, just be aware it is ...|     0|mh_-eMZ6K5RLWhZyISBhwA|
|7ATYjTIgM3jUlt4UM3IypQ|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR9DZGw|  5.0|I've taken a lot of spin classes over the years...|     1|OyoGAe7OKpv6SyGZT5g77Q|
|YjUWPpI6HXG530lwP-fb2A|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67Z4Jig|  3.0|Family diner. Had the buffet. Eclectic 

+-----+-------+
|stars|  count|
+-----+-------+
|  1.0|1069561|
|  2.0| 544240|
|  3.0| 691934|
|  4.0|1452918|
|  5.0|3231627|
+-----+-------+


📝 Text statistics:


+-----------------+----------+----------+
|       avg_length|min_length|max_length|
+-----------------+----------+----------+
|567.7644364746477|         1|      5000|
+-----------------+----------+----------+




💾 Parquet saved: ../data/raw/yelp_reviews_raw.parquet


In [6]:
#@title Sample dataset

# Parquet path
parquet_path = "../data/filtered/yelp_reviews_sentiment.parquet"

if os.path.exists(parquet_path):
    print(f"✅ Already exists: {parquet_path}")
    df_sentiment = spark.read.parquet(parquet_path)
    print(f"🔄 Loaded")
else:
    #  Sample directly from the original dataframe with stars
    # 10% sample = ~700K reviews (still >1GB when processed with text)
    df_sample = df_reviews.sample(fraction=0.20, seed=42)

    # Create sentiment column
    df_sentiment = df_sample.select(
        F.col('review_id'),
        F.col('text'),
        F.col('stars'),
        F.col('useful'),
        F.col('date')
    ).withColumn('sentiment',
        F.when(F.col('stars').isin([1.0, 2.0]), 'negative')
        .when(F.col('stars') == 3.0, 'neutral')
        .when(F.col('stars').isin([4.0, 5.0]), 'positive')
    )

    # Single count operation
    total_reviews = df_sentiment.count()
    print(f"\n✅ Sample dataset created: {total_reviews:,} reviews")

    # Get distribution (single pass)
    print("\n🎯 Sentiment distribution:")
    sentiment_counts = df_sentiment.groupBy('sentiment').count().collect()
    for row in sentiment_counts:
        percentage = (row['count'] / total_reviews) * 100
        print(f"   {row['sentiment']}: {row['count']:,} ({percentage:.1f}%)")

    # Show one sample per sentiment (lightweight)
    print("\n📌 Sample reviews:")
    for sent in ['negative', 'neutral', 'positive']:
        sample = df_sentiment.filter(F.col('sentiment') == sent).select('text', 'stars').first()
        if sample:
            print(f"\n{sent.upper()} ({sample['stars']} stars):")
            print(f"   {sample['text'][:150]}...")

    df_sentiment.write.parquet(parquet_path, mode="overwrite")
    print(f"\n💾 Parquet saved: {parquet_path}")


✅ Sample dataset created: 1,397,347 reviews

🎯 Sentiment distribution:


   positive: 936,359 (67.0%)
   neutral: 138,515 (9.9%)
   negative: 322,473 (23.1%)

📌 Sample reviews:

NEGATIVE (2.0 stars):
   We started with the artichoke dip, big mistake. It was cold and the chips were stale. Mentioned to our waitress, but nothing came of it.  We ordered a...

NEUTRAL (3.0 stars):
   Honestly the food doesn't knock my socks off but other people seem to love this place. I go because my husband likes it as for me I'd rather go to a d...

POSITIVE (5.0 stars):
   Amazingly amazing wings and homemade bleu cheese. Had the ribeye: tender, perfectly prepared, delicious. Nice selection of craft beers. Would DEFINITE...



💾 Parquet saved: ../data/filtered/yelp_reviews_sentiment.parquet


## Data Cleaning

In [7]:
#@title Select relevant features and drop duplicates

df_filtered = df_sentiment.select("text", "sentiment") \
                .dropDuplicates()

df_filtered.show(10)

+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|The pasta was coo...| positive|
|The food is excel...| negative|
|I have only ever ...| negative|
|Although the term...| positive|
|1st time here, my...| positive|
|Cleanliness is an...| negative|
|Some cool stuff.....| positive|
|I don't know why ...|  neutral|
|I love pho and th...| positive|
|My husband and I ...|  neutral|
+--------------------+---------+
only showing top 10 rows



In [8]:
#@title Balance dataset

sentiment_counts = df_filtered.groupBy('sentiment').count().collect()
min_count = min([row['count'] for row in sentiment_counts]) 

print(f"\n🔄 Balancing dataset: {df_filtered.count():,} reviews")

df_negative = df_filtered.filter(F.col('sentiment') == 'negative').limit(min_count)
df_neutral = df_filtered.filter(F.col('sentiment') == 'neutral').limit(min_count)
df_positive = df_filtered.filter(F.col('sentiment') == 'positive').limit(min_count)
df_balanced = df_negative.union(df_neutral).union(df_positive)

print(f"\n✅ Balanced dataset: {df_balanced.count():,} reviews")


🔄 Balancing dataset: 1,396,694 reviews



✅ Balanced dataset: 415,431 reviews


In [9]:
#@title Clean text

# Add text length
df_clean = df_balanced.withColumn('text_length', F.length(F.col('text')))

# Add word count
df_clean = df_clean.withColumn('word_count', 
    F.size(F.split(F.col('text'), ' ')))

# Clean text: lowercase, remove special characters
df_clean = df_clean.withColumn('text_clean',
    F.lower(F.regexp_replace(F.col('text'), '[^a-zA-Z0-9\\s]', ''))
)

print("\n🧹 Text cleaned:")
df_clean.select('text', 'text_clean', 'sentiment').show(3, truncate=80)


🧹 Text cleaned:


+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+---------+
|                                                                            text|                                                                      text_clean|sentiment|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+---------+
|The food is excellent. The customer service and quantity over time has contin...|the food is excellent the customer service and quantity over time has continu...| negative|
|I have only ever gotten take-out at the Bridgeport Rib House, so I can only c...|i have only ever gotten takeout at the bridgeport rib house so i can only com...| negative|
|Cleanliness is an issue here.  It's a great spot for a quick toenail trim but...|cleanliness is an issue here  its a great spot f

In [10]:
#@title Tokenize text

# Parquet path
parquet_path = "../data/clean/yelp_reviews_tokenized.parquet"

# Tokenize text
tokenizer = Tokenizer(inputCol="text_clean", outputCol="tokens")
df_tokenized = tokenizer.transform(df_clean)

# Remove stop words
remover = StopWordsRemover(inputCol="tokens", outputCol="tokens_filtered")
df_tokenized = remover.transform(df_tokenized)

print("\n📝 Sample tokenized text:")
df_tokenized.select('text_clean', 'tokens_filtered').show(5, truncate=80)

if not os.path.exists(parquet_path):
    df_tokenized.write.parquet(parquet_path, mode="overwrite")
    print(f"\n💾 Parquet saved: {parquet_path}")


📝 Sample tokenized text:


+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                      text_clean|                                                                 tokens_filtered|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|the food is excellent the customer service and quantity over time has continu...|[food, excellent, customer, service, quantity, time, continued, go, ive, goin...|
|i have only ever gotten takeout at the bridgeport rib house so i can only com...|[ever, gotten, takeout, bridgeport, rib, house, comment, , numerous, visits, ...|
|cleanliness is an issue here  its a great spot for a quick toenail trim but i...|[cleanliness, issue, , great, spot, quick, toenail, trim, smells, like, dog, ...|
|i really wanted


💾 Parquet saved: ../data/clean/yelp_reviews_tokenized.parquet
